part 1 main dataframe

main libraries import

In [43]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup


In [52]:
#beautiful soup object creating
# we use one of the previous versions of wikipedia page due to better format
URL = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=926287641'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
table=soup.find('table')

defining and appending

In [49]:
table_contents=[]
for row in table.findAll('tr')[1:]:
    cell = {}
    if row.findAll('td')[1:2][0].text=='Not assigned':    
        pass
    else:
        cell['PostalCode'] = row.td.text[:3]
        cell['Borough'] = row.findAll('td')[1:2][0].text.strip("\n")
        
       
        if row.findAll('td')[2:3][0].text == 'Not assigned\n':
            cell['Neighborhood'] = row.findAll('td')[1:2][0].text.strip("\n")
        else:
            cell['Neighborhood'] = row.findAll('td')[2:3][0].text.strip("\n")
        
        table_contents.append(cell)

In [53]:
df=pd.DataFrame(table_contents)

In [56]:
df['Neighborhood'] = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].transform(lambda x: ', '.join(x))
df = df.drop_duplicates()
df = df.reset_index()
df.drop(['index'], axis=1, inplace=True)
print(df.head(10))

  PostalCode           Borough                      Neighborhood
0        M3A        North York                         Parkwoods
1        M4A        North York                  Victoria Village
2        M5A  Downtown Toronto                      Harbourfront
3        M6A        North York  Lawrence Heights, Lawrence Manor
4        M7A  Downtown Toronto                      Queen's Park
5        M9A         Etobicoke                  Islington Avenue
6        M1B       Scarborough                    Rouge, Malvern
7        M3B        North York                   Don Mills North
8        M4B         East York   Woodbine Gardens, Parkview Hill
9        M5B  Downtown Toronto          Ryerson, Garden District


In [57]:
print(df.shape)

(103, 3)


part 2 Toronto coordinations (latitude and longitude)

In [58]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

import geocoder

In [59]:
geoSpatial = pd.read_csv('Geospatial_Coordinates (1).csv')
print(geoSpatial.head(5))

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476


In [61]:
Latitude = []
Longitude = []

for index, row in df.iterrows():
    postal_code = row['PostalCode']
    Latitude.append(geoSpatial.loc[geoSpatial['Postal Code']==postal_code]['Latitude'].iloc[0])    
    Longitude.append(geoSpatial.loc[geoSpatial['Postal Code']==postal_code]['Longitude'].iloc[0])

In [62]:
df['Longitude'] = Longitude
df['Latitude'] = Latitude


 Final Table

In [63]:
print(df.head(10))

  PostalCode           Borough                      Neighborhood  Longitude  \
0        M3A        North York                         Parkwoods -79.329656   
1        M4A        North York                  Victoria Village -79.315572   
2        M5A  Downtown Toronto                      Harbourfront -79.360636   
3        M6A        North York  Lawrence Heights, Lawrence Manor -79.464763   
4        M7A  Downtown Toronto                      Queen's Park -79.389494   
5        M9A         Etobicoke                  Islington Avenue -79.532242   
6        M1B       Scarborough                    Rouge, Malvern -79.194353   
7        M3B        North York                   Don Mills North -79.352188   
8        M4B         East York   Woodbine Gardens, Parkview Hill -79.309937   
9        M5B  Downtown Toronto          Ryerson, Garden District -79.378937   

    Latitude  
0  43.753259  
1  43.725882  
2  43.654260  
3  43.718518  
4  43.662301  
5  43.667856  
6  43.806686  
7  43.7459

Part 3 - Analysis for Toronto

In [64]:
from geopy.geocoders import Nominatim
import folium
from pandas.io.json import json_normalize

In [69]:
address = 'Toronto'
geolocator = Nominatim(user_agent="tr_explorer")
toronto_location = geolocator.geocode(address)
toronto_latitude = toronto_location.latitude
toronto_longitude = toronto_location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(toronto_latitude, toronto_longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


map creating and markers adding

In [71]:
map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto